In [24]:
# COMP30027 Machine learning Project 1: Gaining Information about Naive Bayes
# Author: Jordan Ung <jordanu@student.unimelb.edu.au> [729938]
# Last Modified: 24.03.19

#### GENERAL GUIDE
# Read In And Inspect The Data
# Check for missing value - (1) delete rows with missing values, (2) Impute the missing values with dataset
# Check for anomalously extreme values

# QUESTIONS TO ASK:
# Should I handle datasets with Missing values and ones without differently?
# In tht case do I need to hardcode it or iterate through the whole dataset once
# to find presence of missing values?

# Is hardcoding bad: ->See "?", is it okay if I add another
# Parameter to the function?

In [25]:
# Preprocess takes the name of a file and returns a list of instances
# within that file, with each instance containing a list of attributes
def preprocess(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        # Add each instance to a list to be used later
        for line in file.readlines():
            dataset.append(line.strip().split(','))
    return dataset

data_file = 'anneal.csv'
missing_value_files = ['breast-cancer.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'primary-tumor.csv']
# preprocess(data_file)

In [26]:
# Train takes a list of instances and returns a 2-tuple, containing
# a list of counts representing the class distribution and a list
# of lists of dictionaries representing the conditional probabilities
def train(instance_list):
    data_info = ([], [])
    classes = {}
    # TODO
    # Need to create a list of lists of dictionaries for repping the
    # conditional probabilities of each attribute w.r.t to the classes
    
    for instance in instance_list:
        # Check for missing values and delete rows containing them
        if "?" in instance:
            instance_list.remove(instance)
        class_type = instance[-1]
        
        # Calculate the frequencies of all the class values
        if class_type not in classes:
            classes[class_type] = 1
        else:
            classes[class_type] += 1
    
    # Create a data structure to store all the attribute probabilities w.r.t each class
    num_of_attributes = len(instance_list[0])
    attribute_data = [[]] * num_of_attributes
    for i in range(len(classes)):
        data_info[1].append(attribute_data)

    # Calculate probability of each class value compared to total number of instances
    num_of_instances = len(instance_list)
    for type in classes:
        data_info[0].append(classes[type] * 1.0 / num_of_instances)
        
    # Calculate probability of each value in each attribute for each class
    for i in range(len(num_of_attributes)):
        values = {}
        for j in range(len(num_of_instances))
        
        
    return data_info # Return the 2-tuple
train(preprocess(missing_value_files[0]))
# train(preprocess(data_file))

([0.30324909747292417, 0.6967509025270758],
 [[[], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], []]])

In [19]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instance):
    return # Return a class

In [5]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [6]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    return

[[], [], [], [], [], [], [], [], [], []]
